In [1]:
from google.colab import drive
drive.mount('/content/drive') 
%cd /content/drive/My\ Drive/
!mkdir ru-gpts
%cd ru-gpts/

Mounted at /content/drive
/content/drive/My Drive
/content/drive/My Drive/ru-gpts


In [2]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Writing setup.sh


In [3]:
!sh setup.sh

Cloning into 'apex'...
remote: Enumerating objects: 7885, done.
remote: Total 7885 (delta 0), reused 0 (delta 0), pack-reused 7885
Receiving objects: 100% (7885/7885), 13.97 MiB | 11.58 MiB/s, done.
Resolving deltas: 100% (5387/5387), done.
Checking out files: 100% (295/295), done.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-ymrszo25
Created temporary directory: /tmp/pip-req-tracker-25dissik
Created requirements tracker '/tmp/pip-req-tracker-25dissik'
Created temporary directory: /tmp/pip-install-ggkh2m6o
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-_6caxmvq
  Added file:///content/drive/My%20Drive/ru-gpts/apex to build tracker '/tmp/pip-req-tracker-25dissik'
    Running setup.py (path:/tmp/pip-req-build-_6caxmvq/

In [4]:
!apt-get install llvm-9-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  binfmt-support libffi-dev libllvm9 libpfm4 llvm-9 llvm-9-runtime
  llvm-9-tools python-chardet python-pkg-resources python-pygments python-yaml
  python3-pkg-resources python3-pygments python3-yaml
Suggested packages:
  llvm-9-doc python-setuptools ttf-bitstream-vera python3-setuptools
The following NEW packages will be installed:
  binfmt-support libffi-dev libllvm9 libpfm4 llvm-9 llvm-9-dev llvm-9-runtime
  llvm-9-tools python-chardet python-pkg-resources python-pygments python-yaml
  python3-pkg-resources python3-pygments python3-yaml
0 upgraded, 15 newly installed, 0 to remove and 15 not upgraded.
Need to get 48.8 MB of archives.
After this operation, 287 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python3-yaml amd64 3.12-1build2 [109 kB]
Get:2 http://archive.ubuntu.com/ubuntu bi

In [5]:
!pip install cpufeature

In [6]:
!pip install triton==0.2.3

     |████████████████████████████████| 952kB 8.2MB/s 
  Created wheel for triton: filename=triton-0.2.3-cp36-cp36m-linux_x86_64.whl size=1211870 sha256=93a2f5f0057fe1560b7695b2df5ad2268824bfd3c622529f2c454190f7ecd449
  Stored in directory: /root/.cache/pip/wheels/b5/14/58/53d64f4fe21694a9f1696a86da4e4005d7ab998265b59940b8
Successfully built triton


In [7]:
!DS_BUILD_CPU_ADAM=1 DS_BUILD_SPARSE_ATTN=1 pip install deepspeed==0.3.7

     |████████████████████████████████| 266kB 8.7MB/s 
     |████████████████████████████████| 225kB 14.6MB/s 
     |████████████████████████████████| 112kB 11.2MB/s 
  Created wheel for deepspeed: filename=deepspeed-0.3.7-cp36-cp36m-linux_x86_64.whl size=4187004 sha256=81067fdea7a11a99a103ce76a5cc51330c34275a23fd035e6c19e06364e77e40
  Stored in directory: /root/.cache/pip/wheels/1b/17/24/9fc923860b838c2979b68ebf9aa91e2ac40b9709c635e58247
Successfully built deepspeed


In [8]:
!ds_report

--------------------------------------------------
DeepSpeed C++/CUDA extension op report
--------------------------------------------------
NOTE: Ops not installed will be just-in-time (JIT) compiled at
      runtime if needed. Op compatibility means that your system
      meet the required dependencies to JIT install the op.
--------------------------------------------------
JIT compiled ops requires ninja
ninja .................. [OKAY]
--------------------------------------------------
op name ................ installed .. compatible
--------------------------------------------------
cpu_adam ............... [YES] ...... [OKAY]
fused_adam ............. [NO] ....... [OKAY]
fused_lamb ............. [NO] ....... [OKAY]
sparse_attn ............ [YES] ...... [OKAY]
transformer ............ [NO] ....... [OKAY]
stochastic_transformer . [NO] ....... [OKAY]
utils .................. [NO] ....... [OKAY]
--------------------------------------------------
DeepSpeed general environment info:
tor

In [9]:
import deepspeed.ops.sparse_attention.sparse_attn_op

In [10]:
!git clone https://github.com/sberbank-ai/ru-gpts.git

Cloning into 'ru-gpts'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 339 (delta 49), reused 59 (delta 26), pack-reused 248
Receiving objects: 100% (339/339), 269.36 KiB | 4.99 MiB/s, done.
Resolving deltas: 100% (173/173), done.


In [11]:
!pip install transformers==3.5.1

     |████████████████████████████████| 1.3MB 8.0MB/s 
     |████████████████████████████████| 2.9MB 36.2MB/s 
     |████████████████████████████████| 890kB 49.0MB/s 
     |████████████████████████████████| 1.1MB 59.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=56af5f9b989396c17d6aa6b276086a5078358708040da4e825acf4c8cc3ccdf5
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [12]:
!pip install natsort

In [13]:
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append("ru-gpts/gw")
from generation_wrapper import RuGPT3XL
gpt = RuGPT3XL.from_pretrained("sberbank-ai/rugpt3xl", seq_len=512)

> initializing model parallel with size 1
> initializing model parallel cuda seeds on global rank 0, model parallel rank 0, and data parallel rank 0 with model parallel seed: 3952 and data parallel seed: 1234



Use alternating sparse & dense attention layers


In [ ]:
import os
import csv
import time
import glob
import json

#script_path = os.path.dirname(os.path.realpath(__file__))
script_path = "/content/drive/MyDrive/ru-gpts"

def writeln(filename, str):
    global script_path
    with open(script_path + "//" + filename, 'w', encoding="utf-8") as the_file:
        the_file.write(str)

if not os.path.exists(script_path + '//result'):
    os.makedirs(script_path + '//result')
files = glob.glob('result/*')
for f in files:
    os.remove(f)

print(time.strftime("%d.%m.%Y %H:%M:%S") + " Start")

with open(script_path + '/ru-gpts/keywords.csv', newline='', encoding='utf-8') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';', quotechar='|')
    flag_first = False
    i = 0
    for row in spamreader:
        if not flag_first:
            flag_first = True
            continue
        i = i + 1
        raw_text = row[0]

        text = gpt.generate(
            text=raw_text,
            max_length=100,
            do_sample=True,
            num_beams=10,
            no_repeat_ngram_size=3,
            repetition_penalty=2.
        )
        text = text[0]
        text = text.replace("<|endoftext|>", "")
        text = os.linesep.join([s for s in text.splitlines() if s])
        params = {}
        params['category'] = row[1]                        
        if row[2]:
            params['title'] = row[2]
        text = "params=" + json.dumps(params) + "\n" + text
        writeln("result//result" + str(i) + ".txt", text)
        print(time.strftime("%d.%m.%Y %H:%M:%S") + " result" + str(i) + ".txt Done")



09.02.2021 14:24:15 Start
